In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
pd.set_option('float_format', '{:f}'.format) # show full number

# Data cleaning of overall survey result

In [3]:
df = pd.read_excel (r'../../../Data/raw_survey_data_402.xlsx')
df

,ID,Start time,Completion time,Email,Name,What is your age?,What is your gender?,What is your current employment status?,What is your monthly income range?,Are you a tertiary student?,Where are you currently studying?,Which region of Singapore do you reside?,Which part of the East region do you reside?,Which part of North region do you reside?,Which part of North-east region do you reside?,Which part of West region do you reside?,Which part of Central (CBD) region do you reside?,Which modes of transportation do you often use?,How often do you take these modes of transportation per week?,Please rank your preferred mode of transport,"Based off your ranking choices above, why do you prefer not to take the last 2 least preferred mode of transport?",Under what circumstances will you take your last 2 least preferred mode of transport?,Please rank these factors based on importance when you are deciding your mode of transport.,"On average, how much in total do you spend on public transport per month?","On a scale of 0 - 10, how likely are you to research on the most efficient route to an unfamiliar destination before embarking on it?","On a scale of 0 -10, while travelling on the public transport, how comfortable are you with being around other commuters?","On a scale of 0 - 10, how likely are you to explore a new route to your destination?","On average, how often do you take the bus per week?","On average, how long do you spend waiting for the bus?",When do you choose to take the bus?,The bus is an affordable mode of transport for me,I can easily find a bus stop near me,I feel safe when travelling on the bus,What would stop you from choosing bus as your mode of travel?,Have you ever chartered a bus?,On what occasion would you charter a bus?,"How much are you willing to pay to charter a bus per trip (One-way, 45 seater)?","On average, how often do you take the train per week?","On average, how much time do you spend per trip?",On what occasions do you take the MRT?,Which MRT lines do you usually take?,MRT is an affordable mode of transport for me,I can easily find a MRT station near me,"When I travel by MRT, I do not need to wait too long for the next train",I feel safe when traveling on the train,What would stop you from choosing MRT as your mode of travel?,"On average, how often do you take the taxi per week?","On average, how long do you have to wait to get a taxi?",How do you usually hail your taxi rides?,On what occasions do you take the taxi?,Please rank the company you prefer to use when taking taxi,Taxi is an affordable mode of transport for me,I can easily find a taxi when I need to,I feel safe when traveling on a taxi,Customer service is important to me when traveling by taxi,"Do you ever experienced difficulties finding a ride? Can you briefly describe when and where? (E.g. MRT breakdown, Raffles)",What would stop you from choosing taxi as your mode of travel?,"On average, how often do you use ride-hailing services per week?",On what occasions do you use ride-hailing services?,Which ride-hailing services have you used before?,Please rank your preferred ride-hailing services.,Why do you prefer these ride-hailing services?,Ride-hailing services have affordable prices,I feel safe when traveling via ride-hailing services,Customer service is important to me when traveling with vehicles booked via ride-hailing app,"Do you ever experienced difficulties finding a ride? Can you briefly describe when and where? (E.g. MRT breakdown, Raffles)2",What would stop you from choosing ride-hailing services as your mode of travel?,Do you have a driving license?,Have you rented a car before?,Please rank your preferred car rental companies.,Why do you prefer to use these companies' services?,How often do you rent the car from these companies?,"On average, what is the duration of your rental?",Why would you prefer to rent a car?,Do you ever experienced difficulties renting a car?,"If Yes, please briefly describe the problem.","When renting a car,

Demographic

In [3]:
demographic_df = df.iloc[:, np.r_[0:1, 5:17]]
demographic_df.columns = ['id', 'age', 'gender', 'employment_status', 'monthly_income','tertiary_student', 'instituition', 'region', 'east', 'north', 'north-east', 'west','central']

demographic_df[['east', 'north', 'north-east', 'west','central']] = demographic_df[['east', 'north', 'north-east', 'west','central']].fillna('')
demographic_df['location']=demographic_df['east'].astype(str)+demographic_df['north'].astype(str)+demographic_df['north-east'].astype(str)+demographic_df['west'].astype(str)+demographic_df['central'].astype(str)
demographic_df = demographic_df.drop(columns=['east', 'north', 'north-east', 'west','central'])

demographic_df['employment_status'] = demographic_df['employment_status'].str[:-1]
lst_col = 'employment_status'
demographic_df = demographic_df.assign(**{lst_col:demographic_df[lst_col].str.split(';')})
demographic_df.employment_status = demographic_df.employment_status.fillna('')
employment_status_df = pd.DataFrame(demographic_df.employment_status.values.tolist()).add_prefix('employment_status_')
demographic_df = demographic_df.merge(employment_status_df, left_index=True, right_index=True)
demographic_df.drop('employment_status', axis=1, inplace=True)

demographic_df.head()

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,id,age,gender,monthly_income,tertiary_student,instituition,region,location,employment_status_0,employment_status_1
0,4,22 - 25,Female,Below $500,Yes,SMU,North,Yishun,Full-time Student,None
1,5,22 - 25,Female,Below $500,Yes,SMU,West,Bukit Batok,Employed Part-time,Full-time Student
2,12,22 - 25,Male,Below $500,Yes,SIT,West,Bukit Batok,Full-time Student,None
3,13,18 - 21,Female,Below $500,Yes,SMU,East,Tampines,Full-time Student,None
4,14,22 - 25,Female,Prefer not to say,Yes,SIM,East,Prefer not to say,Contract,None


Overall mode of transport

In [4]:
mot_df = df.iloc[:, 17:27]
mot_df.columns = ['frequently_used_mot', 'frequency_mot', 'preferred_mot', 'reason_least_preferred_mot','circ_least_preferred_mot', 'factors_mot_ranking', 'overall_avg_spending', 'research', 'comfortable', 'explore']

In [5]:
col_name = ['frequently_used_mot', 'preferred_mot', 'factors_mot_ranking']

for i in col_name:
    mot_df[i] = mot_df[i].str[:-1]
    mot_df[i] = mot_df[i].fillna('')
    mot_df[i] = [x.split(';') for x in mot_df[i]]
    
#fu_mot = pd.DataFrame(mot_df.frequently_used_mot.values.tolist()).add_prefix('frequently_used_mot_')
#mot_df = mot_df.merge(fu_mot, left_index=True, right_index=True)
#demographic_df.drop('employment_status', axis=1, inplace=True)

mot_df['frequently_used_mot_cnt'] = mot_df['frequently_used_mot'].str.len()
mot_df['most_preferred_mot'] = mot_df.preferred_mot.apply(lambda x: x[0])
mot_df['least_preferred_mot'] = mot_df.preferred_mot.apply(lambda x: x[-1])

mot_df['most_impt_factor'] = mot_df.factors_mot_ranking.apply(lambda x: x[0])
mot_df['least_impt_factor'] = mot_df.factors_mot_ranking.apply(lambda x: x[-1])

In [6]:
mot_df

,frequently_used_mot,frequency_mot,preferred_mot,reason_least_preferred_mot,circ_least_preferred_mot,factors_mot_ranking,overall_avg_spending,research,comfortable,explore,frequently_used_mot_cnt,most_preferred_mot,least_preferred_mot,most_impt_factor,least_impt_factor
0,"[MRT, Bus]",Once a week,"[Ride-hailing services, MRT, Bus , Taxi, Car r...",Expensive to rent a car. Singapore too hot to ...,When there is no choice,"[Physical Accessibility, Price, Safety, Reliab...",$50-$100,9.000000,9.000000,1.000000,2,Ride-hailing services,"Others: Cycle, Walk, Drive",Physical Accessibility,Service Information
1,"[Bus, MRT, Taxi, Ride-hailing services]",3 - 4 days a week,"[Ride-hailing services, Taxi, Bus , MRT, Other...",Car Rentals are expensive. Destination too far...,If it's nearby or if car rentals are the cheap...,"[Travel Time, Price, Waiting Time, Comfort, Ph...",More than $150,6.000000,3.000000,4.000000,4,Ride-hailing services,Car rentals,Travel Time,Service Information
2,"[Bus, MRT, Ride-hailing services]",Daily,"[Bus , MRT, Ride-hailing services, Others: Cyc...",The price point,In dire situation where I can’t get grab/gojek...,"[Price, Waiting Time, Travel Time, Reliability...",$50-$100,8.000000,7.000000,6.000000,3,Bus,Car rentals,Price,Service Information
3,"[Bus, MRT]",Daily,"[MRT, Bus , Others: Cycle, Walk, Drive, Taxi, ...",Expensive,Running late,"[Price, Travel Time, Comfort, Waiting Time, Ph...",$50-$100,10.000000,10.000000,10.000000,2,MRT,Car rentals,Price,Safety
4,"[MRT, Bus]",3 - 4 days a week,"[Bus , MRT, Taxi, Ride-hailing services, Car r...",NaN,NaN,"[Price, Travel Time, Comfort, Physical Accessi...",$50-$100,7.000000,7.000000,7.000000,2,Bus,"Others: Cycle, Walk, Drive",Price,Customer Service
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,[],NaN,[],NaN,NaN,[],NaN,nan,nan,nan,1,,,,
398,[],NaN,[],NaN,NaN,[],NaN,nan,nan,nan,1,,,,
399,[],NaN,[],NaN,NaN,[],NaN,nan,nan,nan,1,,,,
400,[],NaN,[],NaN,NaN,[],NaN,nan,nan,nan,1,,,,


Bus

In [7]:
bus_df = df.iloc[:, 27:37]
bus_df.columns = ['frequency_bus', 'waiting_time_bus', 'reason_bus', 'affordable_bus','accesible_bus', 'safe_bus', 'prevent_bus', 'chartered_bus', 'occasion_chartered_bus', 'price_chartered_bus']

In [8]:
col_name = ['reason_bus', 'prevent_bus']

for i in col_name:
    bus_df[i] = bus_df[i].str[:-1]
    bus_df[i] = bus_df[i].fillna('')
    bus_df[i] = [x.split(';') for x in bus_df[i]]

In [9]:
bus_df

,frequency_bus,waiting_time_bus,reason_bus,affordable_bus,accesible_bus,safe_bus,prevent_bus,chartered_bus,occasion_chartered_bus,price_chartered_bus
0,None,Less than 10 minutes,"[When travelling to the nearby MRT Stations, D...",Agree,Strongly Agree,Strongly Agree,"[Bulky items, Long waiting time, Long travel t...",No,NaN,I don't know
1,2-4 times a week,10-20 minutes,"[When travelling to the nearby MRT Stations, D...",Agree,Agree,Agree,"[Bulky items, Too crowded]",No,Going on a trip to somewhere with many people ...,$101 - $120
2,Daily,Less than 10 minutes,"[When travelling to the nearby MRT Stations, D...",Agree,Agree,Agree,"[Long waiting time, Bulky items]",No,NaN,< $100
3,2-4 times a week,Less than 10 minutes,"[When travelling to the nearby MRT Stations, D...",Agree,Agree,Agree,"[Long waiting time, Bulky items, Too crowded]",No,NaN,< $100
4,2-4 times a week,Less than 10 minutes,[Direct to recreation destination (e.g. food c...,Neutral,Neutral,Neutral,"[Long travel time, Too crowded]",No,NaN,I don't know
...,...,...,...,...,...,...,...,...,...,...
397,NaN,NaN,[],NaN,NaN,NaN,[],NaN,NaN,NaN
398,NaN,NaN,[],NaN,NaN,NaN,[],NaN,NaN,NaN
399,NaN,NaN,[],NaN,NaN,NaN,[],NaN,NaN,NaN
400,NaN,NaN,[],NaN,NaN,NaN,[],NaN,NaN,NaN


MRT

In [10]:
mrt_df = df.iloc[:, 37:46]
mrt_df.columns = ['frequency_mrt', 'spend_mrt', 'reason_mrt', 'mrt_lines','affordable_mrt', 'accessible_mrt', 'waiting_mrt', 'safe_mrt', 'prevent_mrt']

In [11]:
col_name = ['reason_mrt', 'mrt_lines', 'prevent_mrt']

for i in col_name:
    mrt_df[i] = mrt_df[i].str[:-1]
    mrt_df[i] = mrt_df[i].fillna('')
    mrt_df[i] = [x.split(';') for x in mrt_df[i]]
    
mrt_df['mrt_lines_cnt'] = mrt_df['mrt_lines'].str.len()

In [12]:
mrt_df

,frequency_mrt,spend_mrt,reason_mrt,mrt_lines,affordable_mrt,accessible_mrt,waiting_mrt,safe_mrt,prevent_mrt,mrt_lines_cnt
0,3-4 days per week,45-60 mins,"[To get to work/school, To directly get to rec...",[North-South Line (Red Line)],Agree,Strongly Agree\n,Strongly Agree\n,Agree,[When I’m late and rushing to get to my destin...,1
1,One day per week,30-45 mins,"[To get to work/school, To directly get to rec...","[Downtown Line (Blue Line), North-South Line (...",Agree,Agree,Agree,Agree,"[Bulky items, Long travel time, Too crowded]",3
2,5-6 days per week,30-45 mins,"[To get to work/school, To directly get to rec...","[Downtown Line (Blue Line), East-West Line (Gr...",Agree,Agree,Agree,Agree,[Long travel time],3
3,5-6 days per week,30-45 mins,"[To get to work/school, To directly get to rec...",[Downtown Line (Blue Line)],Agree,Agree,Agree,Agree,[Too crowded],1
4,3-4 days per week,30-45 mins,[To get to work/school],[East-West Line (Green Line)],Neutral,Neutral,Neutral,Neutral,[Long travel time],1
...,...,...,...,...,...,...,...,...,...,...
397,NaN,NaN,[],[],NaN,NaN,NaN,NaN,[],1
398,NaN,NaN,[],[],NaN,NaN,NaN,NaN,[],1
399,NaN,NaN,[],[],NaN,NaN,NaN,NaN,[],1
400,NaN,NaN,[],[],NaN,NaN,NaN,NaN,[],1


Taxi

In [13]:
taxi_df = df.iloc[:, 46:57]
taxi_df.columns = ['frequency_taxi', 'waiting_time_taxi', 'way_hail', 'reason_taxi', 'preference_rank_taxi', 'affordable_taxi', 'accessible_taxi', 'safe_taxi', 'customer_service_taxi', 'difficulties_taxi', 'prevent_taxi']

In [14]:
col_name = ['way_hail', 'reason_taxi', 'preference_rank_taxi', 'prevent_taxi']

for i in col_name:
    taxi_df[i] = taxi_df[i].str[:-1]
    taxi_df[i] = taxi_df[i].fillna('')
    taxi_df[i] = [x.split(';') for x in taxi_df[i]]
    
taxi_df['most_preferred_taxi'] = taxi_df.preference_rank_taxi.apply(lambda x: x[0])
taxi_df['least_preferred_taxi'] = taxi_df.preference_rank_taxi.apply(lambda x: x[-1])

In [15]:
taxi_df

,frequency_taxi,waiting_time_taxi,way_hail,reason_taxi,preference_rank_taxi,affordable_taxi,accessible_taxi,safe_taxi,customer_service_taxi,difficulties_taxi,prevent_taxi,most_preferred_taxi,least_preferred_taxi
0,Never,5-10 mins,"[Flag down, Use grab app ]","[When it is more accessible, When there is a p...","[ComfortDelgro, Trans-Cab, SMRT Taxi, Premier ...",Neutral,Neutral,Agree,Neutral,Yes when it is late at night. From SMU back to...,"[Too costly, Private hire is cheaper]",ComfortDelgro,Yellow-Top Taxi
1,Once a week,5-10 mins,[Online/In-app],"[When there is a promotion, When it is easy to...","[ComfortDelgro, Trans-Cab, CityCab, Prime Taxi...",Disagree,Disagree,Agree,Agree,"Peak hours, CBD or secluded areas","[Too costly, Private hire is cheaper, Long wai...",ComfortDelgro,Yellow-Top Taxi
2,Never,Less than 5 mins,"[Online/In-app, Flag down]",[When the price is cheaper ],"[ComfortDelgro, Trans-Cab, CityCab, SMRT Taxi,...",Disagree,Agree,Strongly Agree,Agree,NaN,"[Too costly, Private hire is cheaper]",ComfortDelgro,Yellow-Top Taxi
3,Never,5-10 mins,"[Flag down, Online/In-app]","[When it is more accessible, When there is a p...","[ComfortDelgro, SMRT Taxi, Premier Taxi, Trans...",Agree,Agree,Agree,Strongly Agree,NIL,[Too costly],ComfortDelgro,Yellow-Top Taxi
4,Never,5-10 mins,[Online/In-app],[When it is more convenient],"[Premier Taxi, ComfortDelgro, Trans-Cab, CityC...",Neutral,Neutral,Neutral,Neutral,-,[Too costly],Premier Taxi,Yellow-Top Taxi
...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,NaN,NaN,[],[],[],NaN,NaN,NaN,NaN,NaN,[],,
398,NaN,NaN,[],[],[],NaN,NaN,NaN,NaN,NaN,[],,
399,NaN,NaN,[],[],[],NaN,NaN,NaN,NaN,NaN,[],,
400,NaN,NaN,[],[],[],NaN,NaN,NaN,NaN,NaN,[],,


Ride-hailing

In [16]:
ride_hailing_df = df.iloc[:, 57:67]
ride_hailing_df.columns = ['frequency_ride_hailing', 'reason_ride_hailing', 'ride_hailing_type', 'preference_rank_ride_hailing', 'reason_ride_hailing_company', 'affordable_ride_hailing', 'safe_ride_hailing', 'customer_service_ride_hailing', 'difficulties_ride_hailing', 'prevent_ride_hailing']

In [17]:
col_name = ['reason_ride_hailing', 'ride_hailing_type', 'preference_rank_ride_hailing', 'reason_ride_hailing_company', 'prevent_ride_hailing']

for i in col_name:
    #ride_hailing_df[i].astype(str)
    ride_hailing_df[i] = ride_hailing_df[i].str[:-1]
    ride_hailing_df[i] = ride_hailing_df[i].fillna('')
    ride_hailing_df[i] = [x.split(';') for x in ride_hailing_df[i]]

ride_hailing_df['used_ride_hailing_svc_cnt'] = ride_hailing_df['ride_hailing_type'].str.len()
ride_hailing_df['most_preferred_ride_hailing'] = ride_hailing_df.preference_rank_ride_hailing.apply(lambda x: x[0])
ride_hailing_df['least_preferred_ride_hailing'] = ride_hailing_df.preference_rank_ride_hailing.apply(lambda x: x[-1])

In [18]:
ride_hailing_df

,frequency_ride_hailing,reason_ride_hailing,ride_hailing_type,preference_rank_ride_hailing,reason_ride_hailing_company,affordable_ride_hailing,safe_ride_hailing,customer_service_ride_hailing,difficulties_ride_hailing,prevent_ride_hailing,used_ride_hailing_svc_cnt,most_preferred_ride_hailing,least_preferred_ride_hailing
0,Once a week,"[When there is a promotion, When it is more ac...","[Grab, Gojek]","[Grab, Gojek, Ryde, Comfort DelGro, Others, Ta...","[Familiar with the software, easy user interfac]",Agree,Agree,Neutral,Nil,"[Poor driving, Long wait time]",2,Grab,Tada Mobility
1,3-4 days a week,"[When it is easy to book, When there is a prom...","[Grab, Gojek, ComfortDelGro, Ryde]","[Grab, Comfort DelGro, Gojek, Ryde, Tada Mobil...",[More affordable and competitive pricing.],Neutral,Agree,Strongly Agree\n,"Peak hours, CBD and secluded areas","[Poor driving, Poor customer service, Too costly]",4,Grab,Others
2,Once a week,"[When it is easy to book, When there is a prom...","[Grab, Gojek, ComfortDelGro]","[Grab, Gojek, Comfort DelGro, Ryde, Tada Mobil...",[Because of ongoing promotion],Neutral,Agree,Strongly Agree\n,NaN,"[Poor driving, Poor customer service]",3,Grab,Others
3,None,"[When there is a promotion, When it is more co...",[Grab],"[Grab, Comfort DelGro, Gojek, Ryde, Tada Mobil...",[],Agree,Neutral,Agree,NaN,"[Too costly, Poor driving, Poor customer service]",1,Grab,Others
4,None,[When it is more convenient],[Grab],"[Grab, Gojek, Ryde, Comfort DelGro, Tada Mobil...",[],Neutral,Neutral,Neutral,NaN,[Too costly],1,Grab,Others
...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,NaN,[],[],[],[],NaN,NaN,NaN,NaN,[],1,,
398,NaN,[],[],[],[],NaN,NaN,NaN,NaN,[],1,,
399,NaN,[],[],[],[],NaN,NaN,NaN,NaN,[],1,,
400,NaN,[],[],[],[],NaN,NaN,NaN,NaN,[],1,,


Car-rental

In [19]:
car_rental_df = df.iloc[:, 67:87]
car_rental_df.columns = ['own_driving_license', 'rent_before', 'preference_rank_rental_car', 'reason_rental_car_company', 'frequency_rental','duration_rental', 'reason_rental_car', 'difficulties_rental_car', 'difficulties_reason_rental_car', 'convenient_rental_car', 'convenient_reason_rental_car', 'affordable_rental_car', 'reasonable_rental_car', 'type_rental_car', 'perception_rental_car', 'max_price_rental_car', 'biz_model_rental', 'biz_model_reason_rental', 'biz_model_important_rental', 'prevent_rental']

In [20]:
col_name = ['preference_rank_rental_car', 'reason_rental_car_company', 'reason_rental_car', 'type_rental_car', 'biz_model_rental', 'biz_model_reason_rental', 'prevent_rental']

for i in col_name:
    #ride_hailing_df[i].astype(str)
    car_rental_df[i] = car_rental_df[i].str[:-1]
    car_rental_df[i] = car_rental_df[i].fillna('')
    car_rental_df[i] = [x.split(';') for x in car_rental_df[i]]
    
car_rental_df['type_rental_car_heard_cnt'] = car_rental_df['type_rental_car'].str.len()

In [21]:
car_rental_df

,own_driving_license,rent_before,preference_rank_rental_car,reason_rental_car_company,frequency_rental,duration_rental,reason_rental_car,difficulties_rental_car,difficulties_reason_rental_car,convenient_rental_car,convenient_reason_rental_car,affordable_rental_car,reasonable_rental_car,type_rental_car,perception_rental_car,max_price_rental_car,biz_model_rental,biz_model_reason_rental,biz_model_important_rental,prevent_rental,type_rental_car_heard_cnt
0,Yes,No,[],[],NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,"[Tribecar, Grabcar Rental, BlueSG]",NaN,$3.01 - $5,[Free-floating],[Convenience],6.000000,"[Inconvenient Pickup Locations, Costly, Safety...",3
1,Yes,Yes,"[BlueSG, No preference, GetGo, Tribecar, Lion...",[Convenient],Once a year,< 1 hour,"[Convenience, Occasions, Too costly to own a c...",No,NaN,No,Near Shopping Malls or MRT stations or Estates...,Strongly disagree,Disagree,[],NaN,NaN,[Free-floating],"[Convenience, Cost]",8.000000,"[Inconvenient Pickup Locations, Costly, Safety...",1
2,Yes,No,[],[],NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,"[BlueSG, GetGO, Tribecar, Grabcar Rental]",NaN,$5.01 - $7,[Station-based],"[Cost, Convenience]",6.000000,"[Inconvenient Pickup Locations, Costly, Insura...",4
3,No,No,[],[],NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,[BlueSG],NaN,$3.01 - $5,[Free-floating],[Convenience],5.000000,"[Costly, Age Restrictions]",1
4,No,No,[],[],NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,[BlueSG],NaN,< $3,"[Station-based, Free-floating]",[Cost],5.000000,[Costly],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,NaN,NaN,[],[],NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,[],[],nan,[],1
398,NaN,NaN,[],[],NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,[],[],nan,[],1
399,NaN,NaN,[],[],NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,[],[],nan,[],1
400,NaN,NaN,[],[],NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,[],[],nan,[],1


Perception of CDG

In [22]:
perception_cdg_df = df.iloc[:, 87:99]
perception_cdg_df.columns = ['aware_cdg', 'used_cdg', 'accessibility_cdg', 'cs_cdg', 'cost_cdg', 'comfort_cdg', 'reliability_cdg', 'info_cdg', 'overall_rating_cdg', 'perception_cdg', 'improvement_cdg', 'incentives_cdg']

In [23]:
col_name = ['aware_cdg', 'used_cdg', 'incentives_cdg']

for i in col_name:
    #ride_hailing_df[i].astype(str)
    perception_cdg_df[i] = perception_cdg_df[i].str[:-1]
    perception_cdg_df[i] = perception_cdg_df[i].fillna('')
    perception_cdg_df[i] = [x.split(';') for x in perception_cdg_df[i]]

perception_cdg_df['aware_cdg_cnt'] = perception_cdg_df['aware_cdg'].str.len()
perception_cdg_df['used_cdg_cnt'] = perception_cdg_df['used_cdg'].str.len()

In [24]:
perception_cdg_df

,aware_cdg,used_cdg,accessibility_cdg,cs_cdg,cost_cdg,comfort_cdg,reliability_cdg,info_cdg,overall_rating_cdg,perception_cdg,improvement_cdg,incentives_cdg,aware_cdg_cnt,used_cdg_cnt
0,[Taxi],[Taxi],Somewhat satisfied,Neutral,Somewhat dissatisfied,Somewhat satisfied,Somewhat satisfied,Neutral,Neutral,Just a taxi company,Not very sure don’t take them often,"[More locations to rent from, Competitive pric...",1,1
1,[Taxi],[Taxi],Neutral,Somewhat dissatisfied,Somewhat dissatisfied,Somewhat satisfied,Somewhat satisfied,Somewhat satisfied,Neutral,NaN,NaN,"[More locations to rent from, Competitive pric...",1,1
2,[Taxi],[Taxi],Somewhat satisfied,Somewhat satisfied,Neutral,Somewhat satisfied,Very satisfied,Somewhat satisfied,Somewhat satisfied,Rides are really comfy,Lowering their price to compete with grab/gojek,"[More locations to rent from, Promotion codes]",1,1
3,[Taxi],[Taxi],Somewhat satisfied,Somewhat satisfied,Somewhat satisfied,Somewhat satisfied,Somewhat satisfied,Somewhat satisfied,Somewhat satisfied,NaN,NaN,[Competitive prices],1,1
4,[Taxi],[Taxi],Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,NaN,NaN,[],1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],1,1
398,[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],1,1
399,[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],1,1
400,[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],1,1


In [25]:
df = pd.concat([demographic_df, mot_df, bus_df, mrt_df, taxi_df, ride_hailing_df, car_rental_df, perception_cdg_df], axis=1)
df

,id,age,gender,monthly_income,tertiary_student,instituition,region,location,employment_status_0,employment_status_1,frequently_used_mot,frequency_mot,preferred_mot,reason_least_preferred_mot,circ_least_preferred_mot,factors_mot_ranking,overall_avg_spending,research,comfortable,explore,frequently_used_mot_cnt,most_preferred_mot,least_preferred_mot,most_impt_factor,least_impt_factor,frequency_bus,waiting_time_bus,reason_bus,affordable_bus,accesible_bus,safe_bus,prevent_bus,chartered_bus,occasion_chartered_bus,price_chartered_bus,frequency_mrt,spend_mrt,reason_mrt,mrt_lines,affordable_mrt,accessible_mrt,waiting_mrt,safe_mrt,prevent_mrt,mrt_lines_cnt,frequency_taxi,waiting_time_taxi,way_hail,reason_taxi,preference_rank_taxi,affordable_taxi,accessible_taxi,safe_taxi,customer_service_taxi,difficulties_taxi,prevent_taxi,most_preferred_taxi,least_preferred_taxi,frequency_ride_hailing,reason_ride_hailing,ride_hailing_type,preference_rank_ride_hailing,reason_ride_hailing_company,affordable_ride_hailing,safe_ride_hailing,customer_service_ride_hailing,difficulties_ride_hailing,prevent_ride_hailing,used_ride_hailing_svc_cnt,most_preferred_ride_hailing,least_preferred_ride_hailing,own_driving_license,rent_before,preference_rank_rental_car,reason_rental_car_company,frequency_rental,duration_rental,reason_rental_car,difficulties_rental_car,difficulties_reason_rental_car,convenient_rental_car,convenient_reason_rental_car,affordable_rental_car,reasonable_rental_car,type_rental_car,perception_rental_car,max_price_rental_car,biz_model_rental,biz_model_reason_rental,biz_model_important_rental,prevent_rental,type_rental_car_heard_cnt,aware_cdg,used_cdg,accessibility_cdg,cs_cdg,cost_cdg,comfort_cdg,reliability_cdg,info_cdg,overall_rating_cdg,perception_cdg,improvement_cdg,incentives_cdg,aware_cdg_cnt,used_cdg_cnt
0,4,22 - 25,Female,Below $500,Yes,SMU,North,Yishun,Full-time Student,None,"[MRT, Bus]",Once a week,"[Ride-hailing services, MRT, Bus , Taxi, Car r...",Expensive to rent a car. Singapore too hot to ...,When there is no choice,"[Physical Accessibility, Price, Safety, Reliab...",$50-$100,9.000000,9.000000,1.000000,2,Ride-hailing services,"Others: Cycle, Walk, Drive",Physical Accessibility,Service Information,None,Less than 10 minutes,"[When travelling to the nearby MRT Stations, D...",Agree,Strongly Agree,Strongly Agree,"[Bulky items, Long waiting time, Long travel t...",No,NaN,I don't know,3-4 days per week,45-60 mins,"[To get to work/school, To directly get to rec...",[North-South Line (Red Line)],Agree,Strongly Agree\n,Strongly Agree\n,Agree,[When I’m late and rushing to get to my destin...,1,Never,5-10 mins,"[Flag down, Use grab app ]","[When it is more accessible, When there is a p...","[ComfortDelgro, Trans-Cab, SMRT Taxi, Premier ...",Neutral,Neutral,Agree,Neutral,Yes when it is late at night. From SMU back to...,"[Too costly, Private hire is cheaper]",ComfortDelgro,Yellow-Top Taxi,Once a week,"[When there is a promotion, When it is more ac...","[Grab, Gojek]","[Grab, Gojek, Ryde, Comfort DelGro, Others, Ta...","[Familiar with the software, easy user interfac]",Agree,Agree,Neutral,Nil,"[Poor driving, Long wait time]",2,Grab,Tada Mobility,Yes,No,[],[],NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,"[Tribecar, Grabcar Rental, BlueSG]",NaN,$3.01 - $5,[Free-floating],[Convenience],6.000000,"[Inconvenient Pickup Locations, Costly, Safety...",3,[Taxi],[Taxi],Somewhat satisfied,Neutral,Somewhat dissatisfied,Somewhat satisfied,Somewhat satisfied,Neutral,Neutral,Just a taxi company,Not very sure don’t take them often,"[More locations to rent from, Competitive pric...",1,1
1,5,22 - 25,Female,Below $500,Yes,SMU,West,Bukit Batok,Employed Part-time,Full-time Student,"[Bus, MRT, Taxi, Ride-hailing services]",3 - 4 days a week,"[Ride-hailing services, Taxi, Bus , MRT, Other...",Car Rentals are expensive. Destination too far...,If it's nearby or if car rentals are the cheap...,"[Travel Time, Price, Waiting Time, Comfort, Ph...",More than $150,6.000000,3.000000,4.0

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402 entries, 0 to 401
Columns: 106 entries, id to used_cdg_cnt
dtypes: float64(4), int64(7), object(95)
memory usage: 333.0+ KB


In [27]:
df.to_csv(r'../../../Data/cleaned_survey_402.csv', index=False)

# Clustering Aspect 1: demographic + usage

In [28]:
clustering_aspect_1 = df[['id' , 'age' , 'gender' , 'monthly_income' ,'tertiary_student' ,'instituition', 'region' ,
                   'location', 'employment_status_0', 'employment_status_1', 'frequently_used_mot', 'frequency_mot', 'preferred_mot', 'factors_mot_ranking', 'overall_avg_spending',
        'frequency_bus', 'waiting_time_bus', 'frequency_mrt', 'spend_mrt', 'mrt_lines',
        'frequency_taxi', 'waiting_time_taxi', 'preference_rank_taxi', 
         'frequency_ride_hailing', 'ride_hailing_type', 'preference_rank_ride_hailing',
        'own_driving_license', 'rent_before', 'preference_rank_rental_car', 'frequency_rental', 'duration_rental',
        'type_rental_car']]

In [29]:
clustering_aspect_1

,id,age,gender,monthly_income,tertiary_student,instituition,region,location,employment_status_0,employment_status_1,frequently_used_mot,frequency_mot,preferred_mot,factors_mot_ranking,overall_avg_spending,frequency_bus,waiting_time_bus,frequency_mrt,spend_mrt,mrt_lines,frequency_taxi,waiting_time_taxi,preference_rank_taxi,frequency_ride_hailing,ride_hailing_type,preference_rank_ride_hailing,own_driving_license,rent_before,preference_rank_rental_car,frequency_rental,duration_rental,type_rental_car
0,4,22 - 25,Female,Below $500,Yes,SMU,North,Yishun,Full-time Student,None,"[MRT, Bus]",Once a week,"[Ride-hailing services, MRT, Bus , Taxi, Car r...","[Physical Accessibility, Price, Safety, Reliab...",$50-$100,None,Less than 10 minutes,3-4 days per week,45-60 mins,[North-South Line (Red Line)],Never,5-10 mins,"[ComfortDelgro, Trans-Cab, SMRT Taxi, Premier ...",Once a week,"[Grab, Gojek]","[Grab, Gojek, Ryde, Comfort DelGro, Others, Ta...",Yes,No,[],NaN,NaN,"[Tribecar, Grabcar Rental, BlueSG]"
1,5,22 - 25,Female,Below $500,Yes,SMU,West,Bukit Batok,Employed Part-time,Full-time Student,"[Bus, MRT, Taxi, Ride-hailing services]",3 - 4 days a week,"[Ride-hailing services, Taxi, Bus , MRT, Other...","[Travel Time, Price, Waiting Time, Comfort, Ph...",More than $150,2-4 times a week,10-20 minutes,One day per week,30-45 mins,"[Downtown Line (Blue Line), North-South Line (...",Once a week,5-10 mins,"[ComfortDelgro, Trans-Cab, CityCab, Prime Taxi...",3-4 days a week,"[Grab, Gojek, ComfortDelGro, Ryde]","[Grab, Comfort DelGro, Gojek, Ryde, Tada Mobil...",Yes,Yes,"[BlueSG, No preference, GetGo, Tribecar, Lion...",Once a year,< 1 hour,[]
2,12,22 - 25,Male,Below $500,Yes,SIT,West,Bukit Batok,Full-time Student,None,"[Bus, MRT, Ride-hailing services]",Daily,"[Bus , MRT, Ride-hailing services, Others: Cyc...","[Price, Waiting Time, Travel Time, Reliability...",$50-$100,Daily,Less than 10 minutes,5-6 days per week,30-45 mins,"[Downtown Line (Blue Line), East-West Line (Gr...",Never,Less than 5 mins,"[ComfortDelgro, Trans-Cab, CityCab, SMRT Taxi,...",Once a week,"[Grab, Gojek, ComfortDelGro]","[Grab, Gojek, Comfort DelGro, Ryde, Tada Mobil...",Yes,No,[],NaN,NaN,"[BlueSG, GetGO, Tribecar, Grabcar Rental]"
3,13,18 - 21,Female,Below $500,Yes,SMU,East,Tampines,Full-time Student,None,"[Bus, MRT]",Daily,"[MRT, Bus , Others: Cycle, Walk, Drive, Taxi, ...","[Price, Travel Time, Comfort, Waiting Time, Ph...",$50-$100,2-4 times a week,Less than 10 minutes,5-6 days per week,30-45 mins,[Downtown Line (Blue Line)],Never,5-10 mins,"[ComfortDelgro, SMRT Taxi, Premier Taxi, Trans...",None,[Grab],"[Grab, Comfort DelGro, Gojek, Ryde, Tada Mobil...",No,No,[],NaN,NaN,[BlueSG]
4,14,22 - 25,Female,Prefer not to say,Yes,SIM,East,Prefer not to say,Contract,None,"[MRT, Bus]",3 - 4 days a week,"[Bus , MRT, Taxi, Ride-hailing services, Car r...","[Price, Travel Time, Comfort, Physical Accessi...",$50-$100,2-4 times a week,Less than 10 minutes,3-4 days per week,30-45 mins,[East-West Line (Green Line)],Never,5-10 mins,"[Premier Taxi, ComfortDelgro, Trans-Cab, CityC...",None,[Grab],"[Grab, Gojek, Ryde, Comfort DelGro, Tada Mobil...",No,No,[],NaN,NaN,[BlueSG]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,387,>29,Female,"$1,001 - $2,000\n",No,NaN,NaN,,others,None,[],NaN,[],[],NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,[],NaN,[],[],NaN,NaN,[],NaN,NaN,[]
398,388,>29,Male,">$3,001",No,NaN,NaN,,Employed Full-time,None,[],NaN,[],[],NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,[],NaN,[],[],NaN,NaN,[],NaN,NaN,[]
399,389,22 - 25,Male,"$2,001 - $3,000",No,NaN,NaN,,Employed Full-time,None,[],NaN,[],[],NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,[],NaN,[],[],NaN,NaN,[],NaN,NaN,[]
400,390,>29,Prefer not to say,"$1,001 - $2,000\n",No,NaN,NaN,,trainee,None,[],NaN,[],[],NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,[],NaN,[],[],NaN,NaN,[],NaN,NaN,[]


In [30]:
columns = ['preferred_mot', 'factors_mot_ranking', 'preference_rank_ride_hailing', 'preference_rank_rental_car', 
           'frequently_used_mot', 'type_rental_car']

clustering_aspect_1['most_preferred_mot'] = clustering_aspect_1.preferred_mot.apply(lambda x: x[0])
clustering_aspect_1['least_preferred_mot'] = clustering_aspect_1.preferred_mot.apply(lambda x: x[-1])

clustering_aspect_1['most_impt_factor'] = clustering_aspect_1.factors_mot_ranking.apply(lambda x: x[0])
clustering_aspect_1['least_impt_factor'] = clustering_aspect_1.factors_mot_ranking.apply(lambda x: x[-1])

clustering_aspect_1['most_preferred_rank_taxi'] = clustering_aspect_1.preference_rank_taxi.apply(lambda x: x[0])
clustering_aspect_1['least_preferred_rank_taxi'] = clustering_aspect_1.preference_rank_taxi.apply(lambda x: x[-1])

clustering_aspect_1['most_preferred_ride_hailing'] = clustering_aspect_1.preference_rank_ride_hailing.apply(lambda x: x[0])
clustering_aspect_1['least_preferred_ride_hailing'] = clustering_aspect_1.preference_rank_ride_hailing.apply(lambda x: x[-1])

clustering_aspect_1['most_preferred_car_rental'] = clustering_aspect_1.preference_rank_rental_car.apply(lambda x: x[0])
clustering_aspect_1['least_preferred_car_rental'] = clustering_aspect_1.preference_rank_rental_car.apply(lambda x: x[-1])

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [31]:
clustering_aspect_1['frequently_used_mot_cnt'] = clustering_aspect_1['frequently_used_mot'].str.len()
clustering_aspect_1['mrt_lines_cnt'] = clustering_aspect_1['mrt_lines'].str.len()
clustering_aspect_1['used_ride_hailing_companies_cnt'] = clustering_aspect_1['ride_hailing_type'].str.len()
clustering_aspect_1['heard_rental_car_cnt'] = clustering_aspect_1['type_rental_car'].str.len()

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [32]:
clustering_aspect_1

,id,age,gender,monthly_income,tertiary_student,instituition,region,location,employment_status_0,employment_status_1,frequently_used_mot,frequency_mot,preferred_mot,factors_mot_ranking,overall_avg_spending,frequency_bus,waiting_time_bus,frequency_mrt,spend_mrt,mrt_lines,frequency_taxi,waiting_time_taxi,preference_rank_taxi,frequency_ride_hailing,ride_hailing_type,preference_rank_ride_hailing,own_driving_license,rent_before,preference_rank_rental_car,frequency_rental,duration_rental,type_rental_car,most_preferred_mot,least_preferred_mot,most_impt_factor,least_impt_factor,most_preferred_rank_taxi,least_preferred_rank_taxi,most_preferred_ride_hailing,least_preferred_ride_hailing,most_preferred_car_rental,least_preferred_car_rental,frequently_used_mot_cnt,mrt_lines_cnt,used_ride_hailing_companies_cnt,heard_rental_car_cnt
0,4,22 - 25,Female,Below $500,Yes,SMU,North,Yishun,Full-time Student,None,"[MRT, Bus]",Once a week,"[Ride-hailing services, MRT, Bus , Taxi, Car r...","[Physical Accessibility, Price, Safety, Reliab...",$50-$100,None,Less than 10 minutes,3-4 days per week,45-60 mins,[North-South Line (Red Line)],Never,5-10 mins,"[ComfortDelgro, Trans-Cab, SMRT Taxi, Premier ...",Once a week,"[Grab, Gojek]","[Grab, Gojek, Ryde, Comfort DelGro, Others, Ta...",Yes,No,[],NaN,NaN,"[Tribecar, Grabcar Rental, BlueSG]",Ride-hailing services,"Others: Cycle, Walk, Drive",Physical Accessibility,Service Information,ComfortDelgro,Yellow-Top Taxi,Grab,Tada Mobility,,,2,1,2,3
1,5,22 - 25,Female,Below $500,Yes,SMU,West,Bukit Batok,Employed Part-time,Full-time Student,"[Bus, MRT, Taxi, Ride-hailing services]",3 - 4 days a week,"[Ride-hailing services, Taxi, Bus , MRT, Other...","[Travel Time, Price, Waiting Time, Comfort, Ph...",More than $150,2-4 times a week,10-20 minutes,One day per week,30-45 mins,"[Downtown Line (Blue Line), North-South Line (...",Once a week,5-10 mins,"[ComfortDelgro, Trans-Cab, CityCab, Prime Taxi...",3-4 days a week,"[Grab, Gojek, ComfortDelGro, Ryde]","[Grab, Comfort DelGro, Gojek, Ryde, Tada Mobil...",Yes,Yes,"[BlueSG, No preference, GetGo, Tribecar, Lion...",Once a year,< 1 hour,[],Ride-hailing services,Car rentals,Travel Time,Service Information,ComfortDelgro,Yellow-Top Taxi,Grab,Others,BlueSG,Lumens Auto,4,3,4,1
2,12,22 - 25,Male,Below $500,Yes,SIT,West,Bukit Batok,Full-time Student,None,"[Bus, MRT, Ride-hailing services]",Daily,"[Bus , MRT, Ride-hailing services, Others: Cyc...","[Price, Waiting Time, Travel Time, Reliability...",$50-$100,Daily,Less than 10 minutes,5-6 days per week,30-45 mins,"[Downtown Line (Blue Line), East-West Line (Gr...",Never,Less than 5 mins,"[ComfortDelgro, Trans-Cab, CityCab, SMRT Taxi,...",Once a week,"[Grab, Gojek, ComfortDelGro]","[Grab, Gojek, Comfort DelGro, Ryde, Tada Mobil...",Yes,No,[],NaN,NaN,"[BlueSG, GetGO, Tribecar, Grabcar Rental]",Bus,Car rentals,Price,Service Information,ComfortDelgro,Yellow-Top Taxi,Grab,Others,,,3,3,3,4
3,13,18 - 21,Female,Below $500,Yes,SMU,East,Tampines,Full-time Student,None,"[Bus, MRT]",Daily,"[MRT, Bus , Others: Cycle, Walk, Drive, Taxi, ...","[Price, Travel Time, Comfort, Waiting Time, Ph...",$50-$100,2-4 times a week,Less than 10 minutes,5-6 days per week,30-45 mins,[Downtown Line (Blue Line)],Never,5-10 mins,"[ComfortDelgro, SMRT Taxi, Premier Taxi, Trans...",None,[Grab],"[Grab, Comfort DelGro, Gojek, Ryde, Tada Mobil...",No,No,[],NaN,NaN,[BlueSG],MRT,Car rentals,Price,Safety,ComfortDelgro,Yellow-Top Taxi,Grab,Others,,,2,1,1,1
4,14,22 - 25,Female,Prefer not to say,Yes,SIM,East,Prefer not to say,Contract,None,"[MRT, Bus]",3 - 4 days a week,"[Bus , MRT, Taxi, Ride-hailing services, Car r...","[Price, Travel Time, Comfort, Physical Accessi...",$50-$100,2-4 times a week,Less than 10 minutes,3-4 days per week,30-45 mins,[East-West Line (Green Line)],Never,5-10 mins,"[Premier Taxi, ComfortDelgro, Trans-Cab, CityC...",None,[Grab],"[Grab, Gojek, Ryde, Comfort DelGro, Tada Mobil...",No,No,[],NaN,NaN,[BlueSG],Bus,"Others: Cycle, Walk, Drive",Price,Customer Se

In [33]:
#clustering_aspect_1.drop(columns, axis=1, inplace=True)
#clustering_df.drop(['preferred_mot', 'factors_mot_ranking', 'preference_rank_ride_hailing'], axis=1, inplace=True)
clustering_aspect_1.to_csv('clustering_aspect_1.csv', index=False)

In [34]:
#df.to_csv('cleaned_survey_380.csv', index=False)

# Clustering Aspect 2:  demographics + behavior + psychographics

In [35]:
clustering_aspect_2 = df[['id' , 'age' , 'gender' , 'monthly_income' ,'tertiary_student' ,'instituition', 'region' ,
                   'location', 'employment_status_0', 'employment_status_1', 'research', 'comfortable', 'explore', 'reason_bus', 'prevent_bus', 'reason_mrt', 'prevent_mrt', 'way_hail', 'reason_taxi', 
             'prevent_taxi', 'reason_ride_hailing', 'prevent_ride_hailing', 'reason_rental_car_company', 
                 'reason_rental_car', 'prevent_rental', 'max_price_rental_car']]

In [36]:
#clustering_aspect_2.to_csv('clustering_aspect_2.csv', index=False)

# Clustering Aspect 3:  demographics + perceptions

In [37]:
clustering_aspect_3 = df[['id' , 'age' , 'gender' , 'monthly_income' ,'tertiary_student' ,'instituition', 'region' ,
                   'location', 'employment_status_0', 'employment_status_1','affordable_bus','accesible_bus','safe_bus', 
                 'affordable_mrt', 'accessible_mrt', 'waiting_mrt', 'safe_mrt', 
                 'affordable_taxi', 'accessible_taxi', 'safe_taxi', 'customer_service_taxi',
                  'affordable_ride_hailing', 'safe_ride_hailing', 'customer_service_ride_hailing',
                  'affordable_rental_car', 'reasonable_rental_car', 'biz_model_rental', 'biz_model_reason_rental',
                  'biz_model_important_rental']]

In [38]:
#clustering_aspect_3.to_csv('clustering_aspect_3.csv', index=False)

# Clustering Analysis

Choose column for analysis and do further cleaning

In [39]:
clustering_df = df[['id','age', 'gender', 'monthly_income', 'tertiary_student', 'instituition', 'region', 'location', 'employment_status_0', 'employment_status_1', 'frequently_used_mot' , 'frequency_mot', 'preferred_mot', 'factors_mot_ranking', 'overall_avg_spending', 'frequency_bus', 'waiting_time_bus', 
                    'frequency_mrt', 'spend_mrt', 'frequency_taxi', 'waiting_time_taxi',
                   'frequency_ride_hailing', 'preference_rank_ride_hailing', 'own_driving_license','rent_before' , 'preference_rank_rental_car', 'frequency_rental', 'duration_rental', 
                    'type_rental_car', 'max_price_rental_car', 'aware_cdg', 'used_cdg']]

In [40]:
columns = ['preferred_mot', 'factors_mot_ranking', 'preference_rank_ride_hailing', 'preference_rank_rental_car', 
           'frequently_used_mot', 'type_rental_car', 'aware_cdg', 'used_cdg']

clustering_df['most_preferred_mot'] = clustering_df.preferred_mot.apply(lambda x: x[0])
clustering_df['least_preferred_mot'] = clustering_df.preferred_mot.apply(lambda x: x[-1])

clustering_df['most_impt_factor'] = clustering_df.factors_mot_ranking.apply(lambda x: x[0])
clustering_df['least_impt_factor'] = clustering_df.factors_mot_ranking.apply(lambda x: x[-1])

clustering_df['most_preferred_ride_hailing'] = clustering_df.preference_rank_ride_hailing.apply(lambda x: x[0])
clustering_df['least_preferred_ride_hailing'] = clustering_df.preference_rank_ride_hailing.apply(lambda x: x[-1])

clustering_df['most_preferred_car_rental'] = clustering_df.preference_rank_rental_car.apply(lambda x: x[0])
clustering_df['least_preferred_car_rental'] = clustering_df.preference_rank_rental_car.apply(lambda x: x[-1])

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [41]:
clustering_df['frequently_used_mot_cnt'] = df['frequently_used_mot'].str.len()
clustering_df['know_rental_car_cnt'] = df['type_rental_car'].str.len()
clustering_df['aware_cdg_cnt'] = df['aware_cdg'].str.len()
clustering_df['used_cdf_cnt'] = df['used_cdg'].str.len()

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [42]:
clustering_df.drop(columns, axis=1, inplace=True)
#clustering_df.drop(['preferred_mot', 'factors_mot_ranking', 'preference_rank_ride_hailing'], axis=1, inplace=True)
clustering_df

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,id,age,gender,monthly_income,tertiary_student,instituition,region,location,employment_status_0,employment_status_1,frequency_mot,overall_avg_spending,frequency_bus,waiting_time_bus,frequency_mrt,spend_mrt,frequency_taxi,waiting_time_taxi,frequency_ride_hailing,own_driving_license,rent_before,frequency_rental,duration_rental,max_price_rental_car,most_preferred_mot,least_preferred_mot,most_impt_factor,least_impt_factor,most_preferred_ride_hailing,least_preferred_ride_hailing,most_preferred_car_rental,least_preferred_car_rental,frequently_used_mot_cnt,know_rental_car_cnt,aware_cdg_cnt,used_cdf_cnt
0,4,22 - 25,Female,Below $500,Yes,SMU,North,Yishun,Full-time Student,None,Once a week,$50-$100,None,Less than 10 minutes,3-4 days per week,45-60 mins,Never,5-10 mins,Once a week,Yes,No,NaN,NaN,$3.01 - $5,Ride-hailing services,"Others: Cycle, Walk, Drive",Physical Accessibility,Service Information,Grab,Tada Mobility,,,2,3,1,1
1,5,22 - 25,Female,Below $500,Yes,SMU,West,Bukit Batok,Employed Part-time,Full-time Student,3 - 4 days a week,More than $150,2-4 times a week,10-20 minutes,One day per week,30-45 mins,Once a week,5-10 mins,3-4 days a week,Yes,Yes,Once a year,< 1 hour,NaN,Ride-hailing services,Car rentals,Travel Time,Service Information,Grab,Others,BlueSG,Lumens Auto,4,1,1,1
2,12,22 - 25,Male,Below $500,Yes,SIT,West,Bukit Batok,Full-time Student,None,Daily,$50-$100,Daily,Less than 10 minutes,5-6 days per week,30-45 mins,Never,Less than 5 mins,Once a week,Yes,No,NaN,NaN,$5.01 - $7,Bus,Car rentals,Price,Service Information,Grab,Others,,,3,4,1,1
3,13,18 - 21,Female,Below $500,Yes,SMU,East,Tampines,Full-time Student,None,Daily,$50-$100,2-4 times a week,Less than 10 minutes,5-6 days per week,30-45 mins,Never,5-10 mins,None,No,No,NaN,NaN,$3.01 - $5,MRT,Car rentals,Price,Safety,Grab,Others,,,2,1,1,1
4,14,22 - 25,Female,Prefer not to say,Yes,SIM,East,Prefer not to say,Contract,None,3 - 4 days a week,$50-$100,2-4 times a week,Less than 10 minutes,3-4 days per week,30-45 mins,Never,5-10 mins,None,No,No,NaN,NaN,< $3,Bus,"Others: Cycle, Walk, Drive",Price,Customer Service,Grab,Others,,,2,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,387,>29,Female,"$1,001 - $2,000\n",No,NaN,NaN,,others,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,,,,,1,1,1,1
398,388,>29,Male,">$3,001",No,NaN,NaN,,Employed Full-time,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,,,,,1,1,1,1
399,389,22 - 25,Male,"$2,001 - $3,000",No,NaN,NaN,,Employed Full-time,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,,,,,1,1,1,1
400,390,>29,Prefer not to say,"$1,001 - $2,000\n",No,NaN,NaN,,trainee,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,,,,,1,1,1,1


In [43]:
clustering_df

,id,age,gender,monthly_income,tertiary_student,instituition,region,location,employment_status_0,employment_status_1,frequency_mot,overall_avg_spending,frequency_bus,waiting_time_bus,frequency_mrt,spend_mrt,frequency_taxi,waiting_time_taxi,frequency_ride_hailing,own_driving_license,rent_before,frequency_rental,duration_rental,max_price_rental_car,most_preferred_mot,least_preferred_mot,most_impt_factor,least_impt_factor,most_preferred_ride_hailing,least_preferred_ride_hailing,most_preferred_car_rental,least_preferred_car_rental,frequently_used_mot_cnt,know_rental_car_cnt,aware_cdg_cnt,used_cdf_cnt
0,4,22 - 25,Female,Below $500,Yes,SMU,North,Yishun,Full-time Student,None,Once a week,$50-$100,None,Less than 10 minutes,3-4 days per week,45-60 mins,Never,5-10 mins,Once a week,Yes,No,NaN,NaN,$3.01 - $5,Ride-hailing services,"Others: Cycle, Walk, Drive",Physical Accessibility,Service Information,Grab,Tada Mobility,,,2,3,1,1
1,5,22 - 25,Female,Below $500,Yes,SMU,West,Bukit Batok,Employed Part-time,Full-time Student,3 - 4 days a week,More than $150,2-4 times a week,10-20 minutes,One day per week,30-45 mins,Once a week,5-10 mins,3-4 days a week,Yes,Yes,Once a year,< 1 hour,NaN,Ride-hailing services,Car rentals,Travel Time,Service Information,Grab,Others,BlueSG,Lumens Auto,4,1,1,1
2,12,22 - 25,Male,Below $500,Yes,SIT,West,Bukit Batok,Full-time Student,None,Daily,$50-$100,Daily,Less than 10 minutes,5-6 days per week,30-45 mins,Never,Less than 5 mins,Once a week,Yes,No,NaN,NaN,$5.01 - $7,Bus,Car rentals,Price,Service Information,Grab,Others,,,3,4,1,1
3,13,18 - 21,Female,Below $500,Yes,SMU,East,Tampines,Full-time Student,None,Daily,$50-$100,2-4 times a week,Less than 10 minutes,5-6 days per week,30-45 mins,Never,5-10 mins,None,No,No,NaN,NaN,$3.01 - $5,MRT,Car rentals,Price,Safety,Grab,Others,,,2,1,1,1
4,14,22 - 25,Female,Prefer not to say,Yes,SIM,East,Prefer not to say,Contract,None,3 - 4 days a week,$50-$100,2-4 times a week,Less than 10 minutes,3-4 days per week,30-45 mins,Never,5-10 mins,None,No,No,NaN,NaN,< $3,Bus,"Others: Cycle, Walk, Drive",Price,Customer Service,Grab,Others,,,2,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,387,>29,Female,"$1,001 - $2,000\n",No,NaN,NaN,,others,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,,,,,1,1,1,1
398,388,>29,Male,">$3,001",No,NaN,NaN,,Employed Full-time,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,,,,,1,1,1,1
399,389,22 - 25,Male,"$2,001 - $3,000",No,NaN,NaN,,Employed Full-time,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,,,,,1,1,1,1
400,390,>29,Prefer not to say,"$1,001 - $2,000\n",No,NaN,NaN,,trainee,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,,,,,1,1,1,1


In [44]:
clustering_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402 entries, 0 to 401
Data columns (total 36 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   id                            402 non-null    int64 
 1   age                           402 non-null    object
 2   gender                        399 non-null    object
 3   monthly_income                399 non-null    object
 4   tertiary_student              399 non-null    object
 5   instituition                  334 non-null    object
 6   region                        340 non-null    object
 7   location                      402 non-null    object
 8   employment_status_0           399 non-null    object
 9   employment_status_1           30 non-null     object
 10  frequency_mot                 340 non-null    object
 11  overall_avg_spending          340 non-null    object
 12  frequency_bus                 340 non-null    object
 13  waiting_time_bus    

In [45]:
clustering_df.columns

Index(['id', 'age', 'gender', 'monthly_income', 'tertiary_student',
       'instituition', 'region', 'location', 'employment_status_0',
       'employment_status_1', 'frequency_mot', 'overall_avg_spending',
       'frequency_bus', 'waiting_time_bus', 'frequency_mrt', 'spend_mrt',
       'frequency_taxi', 'waiting_time_taxi', 'frequency_ride_hailing',
       'own_driving_license', 'rent_before', 'frequency_rental',
       'duration_rental', 'max_price_rental_car', 'most_preferred_mot',
       'least_preferred_mot', 'most_impt_factor', 'least_impt_factor',
       'most_preferred_ride_hailing', 'least_preferred_ride_hailing',
       'most_preferred_car_rental', 'least_preferred_car_rental',
       'frequently_used_mot_cnt', 'know_rental_car_cnt', 'aware_cdg_cnt',
       'used_cdf_cnt'],
      dtype='object')

In [46]:
#cleaned_survey.columns

In [47]:
#clustering_df.to_csv('clustering_df.csv', index=False)

In [48]:
#dataframe.head()

# Splitting columns into different aspects

In [49]:
psycho_col = ['research', 'comfortable', 'explore']

In [50]:
behaviour_col = ['reason_bus', 'prevent_bus', 'reason_mrt', 'prevent_mrt', 'way_hail', 'reason_taxi', 
             'prevent_taxi', 'reason_ride_hailing', 'prevent_ride_hailing', 'reason_rental_car_company', 
                 'reason_rental_car', 'prevent_rental', 'max_price_rental_car']

In [51]:
perception_col = ['affordable_bus','accesible_bus','safe_bus', 'affordable_mrt', 'accessible_mrt', 'waiting_mrt', 'safe_mrt', 
                 'affordable_taxi', 'accessible_taxi', 'safe_taxi', 'customer_service_taxi',
                  'affordable_ride_hailing', 'safe_ride_hailing', 'customer_service_ride_hailing',
                  'affordable_rental_car', 'reasonable_rental_car', 'biz_model_rental', 'biz_model_reason_rental',
                  'biz_model_important_rental']

In [52]:
cdg_perception = [ 'accessibility_cdg' , 'cs_cdg', 'cost_cdg' , 'comfort_cdg' , 'reliability_cdg' , 'info_cdg', 'overall_rating_cdg']

In [54]:
demographic_col = ['id' , 'age' , 'gender' , 'monthly_income' ,'tertiary_student' ,'instituition', 'region' ,
                   'location', 'type_of_student', 'type_of_employment']

In [55]:
preference = ['preferred_mot', 'most_preferred', 'least_repfeered']

In [56]:
travel_pattern = ['frequently_used_mot', 'frequency_mot', 'factors_mot_ranking', 'overall_avg_spending', 'research', 'comfortable', 'explore',
            'frequently_used_mot_cnt', 'frequency_bus', 'waiting_time_bus', 'chartered_bus', 'frequency_mrt', 'spend_mrt', 'mrt_lines', 'mrt_lines_cnt', 
        'frequency_taxi', 'waiting_time_taxi', 'wail_hail'
         'frequency_ride_hailing', 'ride_hailing_type', 'preference_rank_ride_hailing',
        'own_driving_license', 'rent_before', 'frequency_rental', 'duration_rental']

In [57]:
len(travel_pattern)

24